In [6]:
from pre3 import data_pre
from input_data import input_df
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

In [4]:
df = input_df()
df = data_pre(df)

In [ ]:
df.info()

In [ ]:
# 仮説 : 建物の構造によって価格が違うのではないか
import seaborn as sns
pref_list = df["都道府県名"].tolist()
pref_list = set(pref_list)

fig, ax = plt.subplots(7, 7, figsize=(70, 50), sharey=True)
for i, pref in enumerate(pref_list):
    sns.boxplot(
        x='建物の構造', y='取引価格（総額）_log',
        data=df[df.都道府県名==pref].sort_values('建物の構造'),
        ax=ax[divmod(i, 7)]).set(title=pref, xlabel=None, ylim=(2, 10))

# hide excess graphs
for i in range(len(pref_list)-7*7, 0):
  ax[-1, i].axis('off')


plt.show()

In [ ]:
sns.boxplot(x='建物の構造', y='取引価格（総額）_log',data=df.sort_values('建物の構造')).set(xlabel=None, ylim=(2, 10))

In [ ]:
import numpy as np
# 各県の都市部から離れているほうが価格が安い
# 山梨県

pref = "北海道"
area = "小樽市"

data =  df[df["都道府県名"]==pref].reset_index()

main = data[data["市区町村名"]==area]
nmain = data[data["市区町村名"]!=area]

plt.scatter(nmain['面積（㎡）'], nmain['取引価格（総額）_log'], color='black')
plt.scatter(main['面積（㎡）'], main['取引価格（総額）_log'], color='red')
plt.show()


In [ ]:
df = df[["都道府県名", "市区町村名", "面積（㎡）", "取引価格（総額）_log"]]
main_area = []

df["平均価格"] = df["取引価格（総額）_log"] / df["面積（㎡）"]

for pref in pref_list:
    data = df[df["都道府県名"]==pref].groupby('市区町村名').mean()
    i = data["平均価格"].max()
    area = data[data["平均価格"] == i].index[0]
    main_area.append(area)

In [ ]:
len(main_area)

In [ ]:
main_area

In [ ]:
area_axis = pd.read_csv('area_axis.csv')

In [ ]:
!pip install tqdm

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install bs4
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm


def get_lat_lon_from_address(address_l):

    url = 'http://www.geocoding.jp/api/'
    latlons = []
    for address in tqdm(address_l):
        payload = {"v": 1.1, 'q': address}
        r = requests.get(url, params=payload)
        ret = BeautifulSoup(r.content,'lxml')
        if ret.find('error'):
            raise ValueError(f"Invalid address submitted. {address}")
        else:
            lat = ret.find('lat').string
            lon = ret.find('lng').string
            latlons.append([lat,lon])
            time.sleep(5)
    return latlons

main_area_axis = get_lat_lon_from_address(main_area)

In [ ]:
main_dict = dict(zip(pref_list, main_area_axis))

In [ ]:
main_dict

In [ ]:
import json

with open("main_area_dict.json", "w") as tf:
    json.dump(main_dict,tf)

In [7]:
def json_load(path):
    with open(path, mode='r') as f:
        data = json.load(f)
        return data

main_dict = json_load('main_area_dict.json')

In [8]:
main_dict

{'岩手県': ['39.701956', '141.15433'],
 '秋田県': ['39.719967', '140.10348'],
 '滋賀県': ['35.112626', '136.207575'],
 '千葉県': ['35.497882', '140.114867'],
 '愛知県': ['35.160804', '136.907689'],
 '兵庫県': ['34.667381', '135.214988'],
 '大阪府': ['34.661106', '135.494556'],
 '京都府': ['34.992615', '135.756837'],
 '徳島県': ['34.070365', '134.554954'],
 '富山県': ['36.754112', '137.025764'],
 '鳥取県': ['35.501108', '134.235101'],
 '高知県': ['33.558882', '133.531238'],
 '新潟県': ['37.446519', '138.85135'],
 '島根県': ['35.468191', '133.048406'],
 '石川県': ['36.408421', '136.445853'],
 '福岡県': ['33.590342', '130.441447'],
 '宮崎県': ['32.58232', '131.665141'],
 '鹿児島県': ['31.596854', '130.557139'],
 '群馬県': ['36.389467', '139.063413'],
 '広島県': ['34.381432', '132.447902'],
 '青森県': ['40.603063', '140.464054'],
 '長崎県': ['32.750333', '129.877889'],
 '三重県': ['35.034295', '136.664476'],
 '熊本県': ['32.802657', '130.725996'],
 '山梨県': ['35.662133', '138.5683'],
 '香川県': ['34.342542', '134.046541'],
 '北海道': ['43.144873', '141.358562'],
 '福島県'

In [10]:
!pip install geopy

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [11]:
from geopy.distance import geodesic

def area_dis(x):
    
    pref = x["都道府県名"]
    main_axis = main_dict[pref]
    ido = x["緯度"]
    kei = x["経度"]
    m_ido = main_axis[0]
    m_kei = main_axis[1]
    
    tu = (ido, kei)
    mtu = (m_ido, m_kei)
    
    dis = geodesic(tu, mtu).km

    return dis
    

df["都市距離"] = df.apply(lambda x: area_dis(x), axis=1)

In [13]:
t = df[["市区町村名", "都市距離"]]

In [51]:
s = t.groupby('市区町村名').mean()

In [52]:
s

,都市距離
市区町村名,
あきる野市,37.291469
あま市,12.146825
いわき市,86.897730
うるま市,25.619556
さいたま市中央区,3.045105
...,...
鶴岡市,68.882127
鹿児島市,0.000000
鹿屋市,37.064919


In [69]:
a = a[["市区町村名"]]

In [70]:
a["都市距離"] = 69.59576188973001

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
a = a.set_index("市区町村名")

In [76]:
type(s)

pandas.core.frame.DataFrame

In [82]:
s = s.append(a)

In [88]:
s = s.reset_index()

In [89]:
s.to_csv('main_area_dis', index=False)

In [48]:
def area_dis(x):
    
    
    with open('main_area_dict.json', mode='r') as f:
        main_dict = json.load(f)
    
    ido = 35.857784
    kei = 140.139166
    m_ido = 36.396651
    m_kei = 140.534793
    
    tu = (ido, kei)
    mtu = (m_ido, m_kei)
    
    dis = geodesic(tu, mtu).km

    return dis

In [19]:
df_test = pd.read_csv("/tmp/working/dataset/nishika/old_apartment_2020/test.csv", index_col=0)

df_test = data_pre(df_test)

In [62]:
a = df_test[df_test["市区町村名"]=="北相馬郡利根町"]

In [63]:
a

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,緯度,経度,クラスタ平均
ID,,,,,,,,,,,,,,,,,,,,
8076952,茨城県,北相馬郡利根町,大字布川,布佐,24.0,３ＬＤＫ,70.0,28.0,ＲＣ,NaN,住宅,第１種中高層住居専用地域,60.0,200.0,2020.75,未改装,NaN,35.857784,140.139166,0.0


In [67]:
area_dis(a)

69.59576188973001

In [68]:
a

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,緯度,経度,クラスタ平均
ID,,,,,,,,,,,,,,,,,,,,
8076952,茨城県,北相馬郡利根町,大字布川,布佐,24.0,３ＬＤＫ,70.0,28.0,ＲＣ,NaN,住宅,第１種中高層住居専用地域,60.0,200.0,2020.75,未改装,NaN,35.857784,140.139166,0.0
